## Background Information
---

This information has been deleted by the IronHacks Team

## Setup
---

In [ ]:

%logstop
%logstart -t -r -q ipython_command_log.py global

#- IRONHACKS RESEARCH TRACKING CODE
#----------------------------------
# The following code is used to help our research team understand how you 
# our notebook environment. We do not collect any personal information with
# the following code, it is used to measure when and how often you work on
# your submission files.

import os
from datetime import datetime
import IPython.core.history as history

ha = history.HistoryAccessor()
ha_tail = ha.get_tail(1)
ha_cmd = next(ha_tail)
session_id = str(ha_cmd[0])
command_id = str(ha_cmd[1])
timestamp = datetime.utcnow().isoformat()
history_line = ','.join([session_id, command_id, timestamp]) + '\n'
logfile = open(os.environ['HOME']+'/ipython_session_log.csv', 'a')
logfile.write(history_line)
logfile.close()

In [ ]:

!python3 -m pip install pandas

### Imports

In [ ]:

import csv
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import statsmodels.api as sm 
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import FunctionTransformer
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics

In [ ]:
# CONFIGURE THE BIGQUERY SETTINGS

BIGQUERY_PROJECT = 'ironhacks-covid19-data'
BIGQUERY_KEYPATH = 'service-account.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = BIGQUERY_KEYPATH
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [ ]:
def printexcel(result, filename):
    frame = pd.DataFrame(list(result.items()), columns=['poi_id','raw_visit_counts'])
    frame['raw_visit_counts'] =  frame['raw_visit_counts'].apply(lambda x: str(x).replace('[','').replace(']',''))
    frame['raw_visit_counts'] = frame['raw_visit_counts'].astype(float)
    frame.to_csv(filename, index=False)
def printexcellessprocessing(frame, filename):
    frame['raw_visit_counts'] = frame['raw_visit_counts'].apply(lambda x: str(x).replace('[','').replace(']',''))
    frame['raw_visit_counts'] = frame['raw_visit_counts'].astype(float)
    frame.to_csv(filename, index=False)

## Query Data
---

In [ ]:

#Linear regression all the data
#right join means only poi_ids that we need
query1 = """
SELECT prediction.poi_id, weekly.location_name, weekly.raw_visit_counts, weekly.week_number
FROM `ironhacks_covid19_competition`.`weekly_patterns` as weekly
right join `ironhacks_covid19_competition`.`prediction_list_poi` as prediction
on weekly.poi_id = prediction.poi_id
"""

query_job1 = bigquery_client.query(query1)
query_result_all = query_job1.to_dataframe()
print(query_result_all)

In [ ]:

#Linear regression data from week 34 onward looks much more apetizing
#right join means only poi_ids that we need
query2 = """
SELECT prediction.poi_id, weekly.raw_visit_counts, weekly.week_number
FROM `ironhacks_covid19_competition`.`weekly_patterns` as weekly
right join `ironhacks_covid19_competition`.`prediction_list_poi` as prediction
on weekly.poi_id = prediction.poi_id
where week_number >= 34
"""

query_job2 = bigquery_client.query(query2)
query_result_34 = query_job2.to_dataframe()
print(query_result_34.head())

In [ ]:

query3 = """
SELECT prediction.poi_id, weekly.location_name, weekly.week_number, weekly.visits_concentration, weekly.raw_visit_counts, weekly.distance_from_home, weekly.median_dwell
FROM `ironhacks_covid19_competition`.`weekly_patterns` as weekly
right join `ironhacks_covid19_competition`.`prediction_list_poi` as prediction
on weekly.poi_id = prediction.poi_id
"""

query_job3 = bigquery_client.query(query3)
query_result_weekly = query_job3.to_dataframe()
print(query_result_weekly.head())

In [ ]:

query4 = """
SELECT week_number, cases, deaths
FROM `ironhacks_covid19_competition`.`covid19_cases`
where county = 'Tippecanoe'
"""

query_job4 = bigquery_client.query(query4)
query_result_covid = query_job4.to_dataframe()
print(query_result_covid)

## Graphs just to visualize some data
---

In [ ]:
#Plotted the data across all locations to see the jist of it
data_to_graph = pd.DataFrame(query_result_all, columns=['poi_id','raw_visit_counts', 'week_number'])

In [ ]:
data_to_graph.plot(x ='week_number', y='raw_visit_counts', kind = 'line')
plt.show()

In [ ]:
#I am just curious what the two locations that spiked are
query_result_alltemp = query_result_all[query_result_all['raw_visit_counts'] > 6000]
print(query_result_alltemp['location_name'])

In [ ]:
query_result_covid.plot(x ='week_number', y='cases', kind = 'line')
plt.show()

In [ ]:
query_result_covid.plot(x ='week_number', y='deaths', kind = 'line')
plt.show()

In [ ]:
#I did some research and in week 33 they reclassified covid deaths to need an official test 
#so 98 deaths went to 13 deaths according to IN.gov. If you extend the rate of of probable deaths 
#(7.5 times (=98/13) as many added after the change ) we 
#should have almost 350 in tippecanoe county, instead of the 42 listed on the official government webpage. 

## Trying to see if there is any colinearity among some likely suspects
---

In [ ]:
#hmm bummer nothing to work with here or else I would have nabbed some coefficients and went to work for multiple regression
#or some machine learning algo
corr = query_result_weekly.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True), square=True)

## Model 1, Linear Regression
---

In [ ]:
#linear regression just to see how bad it is, looks kinda linear bar Purdue??
def regression(data, yvar, xvars):
    result = sm.OLS(data[yvar], data[xvars]).fit()
    return result.predict(44)

In [ ]:
#r squared value average for all the models, not bad tbh
def regressionsquare(data, yvar, xvars):
    result = sm.OLS(data[yvar], data[xvars]).fit()
    return result.rsquared
result = query_result_all.groupby('poi_id').apply(regressionsquare, 'raw_visit_counts', 'week_number')
print(result.mean())

In [ ]:
result_dict1 = query_result_all.groupby('poi_id').apply(regression, 'raw_visit_counts', 'week_number').to_dict()
printexcel(result_dict1, 'model1.csv')

## Model 2, Linear Regression after week 33
---

In [ ]:
#r squared value average for all the models past week 33, jeesh pretty decent might end up going with this guy for accuracy
def regressionsquare(data, yvar, xvars):
    result = sm.OLS(data[yvar], data[xvars]).fit()
    return result.rsquared
result = query_result_34.groupby('poi_id').apply(regressionsquare, 'raw_visit_counts', 'week_number')
print(result.mean())

In [ ]:
result_dict2 = query_result_34.groupby('poi_id').apply(regression, 'raw_visit_counts', 'week_number').to_dict()
printexcel(result_dict2, 'model2.csv')

## Model 3, try to find better models for bad guys
---

In [ ]:
#my logic is these might not be linear or need some parameter adjusting or forced intercept
#this module shows how many are not happy with the R2 value
#'result' variable is from either model 1 or model 2
resultdict = result.to_dict()
badmodels = dict()
verybadmodels = dict()
veryverybadmodels = dict()
sum = 0
for (key, value) in resultdict.items():
    if value < .8: #and value > .7: #decided I wanted to look at all of these instead, used to be .9, now .8
        badmodels[key] = value
    if value < .7 and value > .5:
        verybadmodels[key] = value
    if value < .5:
        veryverybadmodels[key] = value
    sum = sum + value
print(len(badmodels))
print(len(verybadmodels))
print(len(veryverybadmodels))
print(sum/len(resultdict))

# Model 3a, exponential models
---

In [ ]:
#I revisted this with query_result_34, which chops off older data before week 34
#'result_dict2' variable is from model 2, change to 'result_dict1' if model 1 is used 
frame = pd.DataFrame(list(result_dict2.items()), columns=['poi_id','raw_visit_counts'])
pois = list(badmodels)
#----------------------------------------------
#Iteration 1, query_result_all
#Trying for exponential fit, only found 1 poi_id above .9 fit.. (.98 4 data points) maybe in the future 
#if I chop off some data it will fit more, ignoring the 1 model for now
#I need a cutoff or some type of dampening for values so the predictions are not too high
#I graphed (veryverybad) guys and they look very much e^(-x)=y or e^(x)=y
#----------------------------------------------
#Iteration 2, query_result_34
# NOW WE'RE IN BUSINESS
# these are decent results for a varity of poi's. The estimate values are very solid by visual inspection
# After deciding these are trustworthy I am lowering the bar to .8
#----------------------------------------------
count =0
for i in range(len(pois)):
    #data that did not model well
    test = query_result_34.loc[query_result_34['poi_id'] == pois[i]]
    
    Y = test['raw_visit_counts']
    X = test['week_number']
    Y = np.stack(Y.values).reshape(-1,1)
    X = np.stack(X.values).reshape(-1,1)

    transformer = FunctionTransformer(np.log, validate=True)
    Y1 = transformer.fit_transform(Y) 
    regressor = LinearRegression()
    results = regressor.fit(X, Y1) 
    r2 = regressor.score(X, Y1)
    yfit = results.predict([[44]])
    predict44 = np.exp(yfit)
    if r2 > .8:
        #print(r2)
        #print(resultdict[pois[i]])
        #print(predict44)
        #print('\n')
        #test.plot(x ='week_number', y='raw_visit_counts', kind = 'scatter')
        #plt.show()
        
        resultdict[pois[i]] = r2 #update r2 value
        #set the predicted value of that model here
        frame.loc[frame['poi_id'] == pois[i], ['raw_visit_counts']] = predict44
        count =count+1
print(count)

In [ ]:
#don't forge to run this in between. I decided I like exponential fit so if it meets the criteria, I want to prefer it.
#now we have a new list of bad models!
badmodels = dict()
verybadmodels = dict()
veryverybadmodels = dict()
count = 0
for (key, value) in resultdict.items():
    if value < .8:
        badmodels[key] = value
    if value < .7 and value > .5:
        verybadmodels[key] = value
    if value < .5:
        veryverybadmodels[key] = value
    count=count+value
print(len(badmodels))
print(len(verybadmodels))
print(len(veryverybadmodels))
print(count/len(resultdict))
#there is a slight improvement in the mean R^2 value of all the models, only slight- we only fixed 25

## Model 3b, polynomial models (d=2,3)
---

In [ ]:
#I revisted this with query_result_34, which chops off older data before week 34
pois = list(badmodels)
#----------------------------------------------
#Iteration 1, query_result_all
#----------------------------------------------
#trying polynomial fit degree 2, only found a few results with poor estimations, not surprising,
#seems like a silly model given the context of the problem
#trying polynomial fit degree 3
#DANGER zone? 4 point data sets R2=1... Perfect?lol But obviously wayyy overfit- Not sure how to handle that.
#Update on DANGER zone: This creates horrible predictions, not at all what I want
#----------------------------------------------
#Iteration 2, query_result_34
#----------------------------------------------
#This is better but I am worried about overfitting, clearly based on the graphs some of them fit well
#but the predictions are inconsistent and I'm worried it would throw my error off.
#I am going to look at fits of .95 or more only, exclude those with only 4 points
#----------------------------------------------
#Iteration 3, query_result_34, after I applied exponential fit to 25 models
#----------------------------------------------
# Not looking too good, only 2 models and 1 of them gives a poor estimation. ugh. I am going to apply
#this to the 1 though because its a sexy X^3 fit
#----------------------------------------------
count = 0
for i in range(len(pois)):
    #data that did not model well
    test = query_result_34.loc[query_result_34['poi_id'] == pois[i]]
    
    Y = test['raw_visit_counts']
    X = test['week_number']
    Y = np.stack(Y.values).reshape(-1,1)
    X = np.stack(X.values).reshape(-1,1)
    
    poly = PolynomialFeatures(degree=3)
    X1 = poly.fit_transform(X)
    Y1 = poly.fit_transform([[44]])

    clf = linear_model.LinearRegression()
    clf.fit(X1, Y)
    predict44 = clf.predict(Y1)
    r2 = clf.score(X1, Y)
    if r2 > .95 and r2 != 1 and pois[i] != '30d6715f-78a3-4ee7-b4ca-fea7706ff9e8': #This guy was a bad fit
        #print(r2)
        #print(resultdict[pois[i]])
        #print(predict44)
        #print('\n')
        #test.plot(x ='week_number', y='raw_visit_counts', kind = 'scatter')
        #plt.show()
        resultdict[pois[i]] = r2 #update r2 value
        #set the predicted value of that model here
        frame.loc[frame['poi_id'] == pois[i], ['raw_visit_counts']] = predict44
        count = count + 1
print(count)
#'1' is the loneliest number, but the sexiest fit

In [ ]:
#updated resultdict results
badmodels = dict()
verybadmodels = dict()
veryverybadmodels = dict()
count = 0
for (key, value) in resultdict.items():
    if value < .8: #and value > .7: #decided I wanted to look at all of these instead, used to be .9, now .8
        badmodels[key] = value
    if value < .7 and value > .5:
        verybadmodels[key] = value
    if value < .5:
        veryverybadmodels[key] = value
    count=count+value
print(len(badmodels))
print(len(verybadmodels))
print(len(veryverybadmodels))
print(count/len(resultdict))
#only fixed 1 additional model, so really no change

In [ ]:
printexcellessprocessing(frame, 'submission_prediction_output.csv')